In [ ]:
# Install necessary libraries
!pip install transformers datasets scikit-learn pandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cu

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ProjectData.csv to ProjectData.csv


In [ ]:
import pandas as pd

# Try common encodings like 'ISO-8859-1' or 'latin1' instead of default 'utf-8'
df = pd.read_csv("ProjectData.csv", encoding='ISO-8859-1')

# Display the first few rows
df.head()
print(df.columns)


Index(['Complaint ID', 'Complaint Text', 'Department'], dtype='object')


In [ ]:
#Strip leading/trailing spaces from column names
df.columns = df.columns.str.strip()
print(df.columns.tolist())

['Complaint ID', 'Complaint Text', 'Department']


In [ ]:
# Rename columns to standard lowercase with underscores
df.rename(columns={
    'Complaint ID': 'complaint_id',
    'Complaint Text': 'complaint_text',
    'Department': 'department'
}, inplace=True)

# Double-check renamed columns
print(df.columns.tolist())


['complaint_id', 'complaint_text', 'department']


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label'] = le.fit_transform(df['department'])
label_names = list(le.classes_)

# Keep only relevant columns
df = df[['complaint_text', 'label']]
df = df.dropna()
df.head()

,complaint_text,label
0,The street lights are not working properly,0
1,Garbage collection has been irregular for a week,4
2,Water supply has been disrupted for two days,6
3,The road near my house has several potholes,2
4,The traffic signals at the intersection are fa...,5


In [ ]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['complaint_text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42
)

In [ ]:
from transformers import BertTokenizer

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch

class ComplaintDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

train_dataset = ComplaintDataset(train_encodings, train_labels)
val_dataset = ComplaintDataset(val_encodings, val_labels)


In [ ]:
from transformers import BertForSequenceClassification

num_labels = len(set(df['label']))
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install transformers --upgrade  # Upgrade transformers to latest version

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    # Replace 'evaluation_strategy' with 'eval_steps' or remove it entirely if unnecessary
    # evaluation_strategy="epoch",
    eval_steps = 100, # Evaluate every 100 steps. Adjust as needed.
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=None,  # Disable logging directory
    report_to="none",  # Disable logging to WandB, TensorBoard, etc.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=78, training_loss=1.4427954844939404, metrics={'train_runtime': 23.3484, 'train_samples_per_second': 53.194, 'train_steps_per_second': 3.341, 'total_flos': 14680405907100.0, 'train_loss': 1.4427954844939404, 'epoch': 3.0})

In [ ]:
# Evaluate the model on the validation dataset
trainer.evaluate()

{'eval_loss': 1.1223810911178589,
 'eval_runtime': 0.2253,
 'eval_samples_per_second': 461.578,
 'eval_steps_per_second': 31.068,
 'epoch': 3.0}

In [ ]:
import torch

# Move the model to the correct device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to predict the department for a given complaint text
def predict_department(complaint_text):
    inputs = tokenizer(complaint_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Forward pass
    outputs = model(**inputs)

    # Get the predicted department
    predicted = torch.argmax(outputs.logits, dim=1)
    return label_names[predicted.item()]

# Get custom input from the user
complaint = input("Enter a complaint text: ")

# Print the predicted department
print(f"Predicted Department: {predict_department(complaint)}")

Enter a complaint text: the potholes on pipestreet are getting worse.
Predicted Department: Sanitation


In [ ]:
# Save the trained model and tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json')

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained("./trained_model")
tokenizer = BertTokenizer.from_pretrained("./trained_model")